# Introduction

# Approach
- model은 input으로 $x_1, x_2, ... , x_n$와 질문인 query $q$를 받음
- model은 output으로 answer $a$를 반환함.
- 각 $x_i, q, a$는 dictionary로 부터 symbol이다.
- model은 모든 $x$를 고정된 buffer size로 memory에 쓰고 그리고 x와 q를 continuous representation을 찾는다.
- continuous representation은 multiple hops로 처기되어 answor $a$로 출력됨.
- 이를 통해 model은 train동안에 여러 memory access를 통해 error signal를 다시 input까지 backpropagation을 할 수 있게함.

## Single Layer
<img src="./images/single_layer.png">

- Input memory representation :  
    - input : $x_1, x_2, ... x_i$   
    - query : $q$
    - input embedding matrix: A  (d x V)  
    - question embedding matrix: B (d x V) 
    - memory vector : $m_i  = \sum_i Ax_i$
    - internal state : $u=Cq$
    - $p_i = Softmax(u^Tm_i)$
    
- output memory representation : 
    - input : $x_1, x_2, ... x_i$   
    - embedding matrix : C (d x V)
    - output vector : $c_i = \sum_i Cx_i$
    - response vector : $o= \sum_i p_ic_i$
 
- Generating the final prediction : 
    - prediction matrix : W (of size V x d)
    - prediction : $\hat{a} = Softmax(W(o+u))$
    
## Multiple Layers
<img src="./images/multi-hop.png">

k layer(k hop)를 쌓는 방법을 알아보자.
- layer의 입력으로 들어오는 것은 이전 layer의 output인 $ok$와 input인 $uk$이다.
$$u^{k+1} = u^k + o^k$$
- 각 layer들은 각자의 embedding matrix인 $A^k, C^k$를 가진다.
- network의 top에는 W를 결합한다.
$$ \hat{a} = Softmax(W(u^{k+1}) = Softmax(W(o^k+u^k))$$


### model에서 weight의 종류
1. Adjacent
    - 앞 layer의 output embedding을 뒷 레이어의 input embedding으로 사용하는 것임. 즉 $A^{k+1}=C^k$
    - 마지막 layer의 prediction matrix $W$는 마지막 output embedidng과 같음. 즉 $W^T= C^k$
    - question embedding은 첫 layer의 iput embedding과 같음. 즉 $B=a^1$
    
2. Layer-wise(RNN-like)
    - input embedding과 output embedding이 다른 layer에서도 동일함. 즉 $A^1=A^2=...=A^k$, $C^1=C^2=...=C^k$
    - H라는 linear mapping을 이용해 update함. $u^{k+1}=Hu^k+o^k$ (H 또한 학습함)
    
우리 모델은 전체적으로 이전 memory network와 유사하다. 하지막 이전 네트워크에서 사용한 hard max operation들을 제외하고 각 layer에서 softmax로 연속적인 가중치로 대체했다.
모델은 p(attention weight)를 

# Synthetic Question and Answering Experiments
학습은 J. Weston, A. Bordes, S. Chopra, and T. Mikolov.  Towards AI-complete question answering:A set of prerequisite toy tasks에서 사용한 데이터를 이용함. 주어진 QA task는 statement의 set으로 구성되고 질문에 대한 답으로는 single word임. 20가지의 task가 있음
<img src="./images/data_sample.png">

주목할 점은 각 question에서 일부 진술만 필요한 정보를 담고 있다. 기존 Memory network에서는 훈련 중에 모델에서 관련이 있는 진술인지 아닌지를 명시적으로 알려줘야 하지만, 이 모델에서는 더 이상 이러한 정보를 제공하지 않아도 된다. 모델이 training 또는 test할 때 스스로 추론해낸다.

## Model details
모든 실험은 adjacent방법과 k=3 hops 인 모델을 사용함. 답변이 여러 단어인 경우 가능한 output에 대해  가능한 조합으로 answer vocabulary word를 만들었음.

- Sentence Representation : 
    1. bag-of-words (BoW) representation 
        - 각 단어들을 embedding한 vector를 sum함.
        - ex $m_i=  \sum_j A x_{ij}$
        - 이 방법의 약점은 sentenc내에서 word의 순서를 알 수 없다는 것임.
        
    2. encodes the posiion of wrds within the sentence
        - 이 방법의 식은 다음과 같음
        - $m_i = \sum_j l_j \cdot A x_ij$
        - column vector인 $l_j$는 다음과 같다
        - $l_kj = (1-j/J) - (k/d)(1-2j/J)$ (J는 sentence의 word의 수, d는 embedding dimension임.)
        - 이러한 방법을 position encoding이라함.
        - memory input만 아니라 memory output, question에도 사용함.
        
- Temporal Encoding :
    - QA task는 temporal context의 개념을 필요할 때가 있음. 
    - 예를들면 sam is in the bedroom after he is in the kitchen과 같은 내용을 이해해야할 수도 있음.
    - 이 모델에서는 memory vector를 $m_i = \sum_j Ax_{ij} + T_A(i)$로 수정해서 이 문제를 해결하려함.
    - $T_A(i)는 temporal information이 encode된 특별한 matrix $T_A$의 i번째 row이다.
    - output embedding에서는  $c_i = \sum_j Cx_{ij} + T_C(i)$이다.
    - $T_A와 T_C$는 training을 통해 학습되며 sharing 제약을 받는다.
    - 문장은 x1이 이야기의 마지막 문장이되도록 질문과의 상대적인 거리를 반영하여 역순으로 색인됩니다.(?)
    
- Learning time invariance by injecting random noise :
    - $T_A$를 정규화하기 위해 'dummy' memory들을 추가하는게 도움이 됨.
    - training할 때 랜덤으로 10%의 빈 memory를 story에 추가함. 
    - 이 접근법을 random noise라함
    
    
# Results
- PE를 한것이 성능이 더 좋음.
- Random noise는 performance향상에 도움이 된다. 특히 training set이 1k보다 작을 떄 도움이 많이 됨.
- joint training은 모든 task에 도움이 된다.
- multiple hop은 performance 향상을 가져온다.
<img src="./images/QA_result.png">

<img src="./images/inference.png">
Support는 Memory Network에서 지원되는 label임. MemN2N에서는 사용하지 않음.
hop에 대한 p를 inference한 것임.
